In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Associated Legendre functions

Play with the recurrence relations for evaluating associated Legendre functions to see how they perform.
Starting from $P_0(x)=1$ and $P_1(x)=x$ we will make use of recurrence relations.  First
$$
  (\ell-m) P_{\ell}^m(x) = (2\ell-1) x P_{\ell-1}^m(x) - (\ell+m-1) P_{\ell-2}^m(x)
$$
is upwards stable for $m=0$.  The $m>0$ functions can be evaluated using a combination of two recurrence relations.  First
$$
    P_m^m(x) = -(2m-1)\sqrt{1-x^2}\ P_{m-1}^{m-1}(x)
$$
and then
$$
    P_{m+1}^m = (2m+1)x P_m^m(x)
$$
You fill in the $\ell > m+1$ relations with the first recurrence relation, building up from $m=0$ (and doing all $\ell$ for each $m$).

In [2]:
xx = np.array([0.025,0.3,0.5,0.7,0.975])
sx = np.sqrt(1-xx**2)
Nx = len(xx)
print(xx)

[0.025 0.3   0.5   0.7   0.975]


In [3]:
# Set up an array for P_ell^m(x).
Nl  = 100
Plm = np.zeros( (Nl,Nl,Nx) )

In [4]:
# Now fill it in. First we do the m=0 case.
Plm[0,0,:] = np.ones_like(xx)
Plm[1,0,:] = xx.copy()
for ell in range(2,Nl):
    Plm[ell,0,:] = (2*ell-1)*xx*Plm[ell-1,0,:]-(ell-1)*Plm[ell-2,0,:]
    Plm[ell,0,:]/= float(ell)

In [5]:
# Now we fill in m>0.
# To keep the recurrences stable, we treat "high m" and "low m" separately.
# Start with the highest value of m allowed:
for m in range(1,Nl):
    Plm[m,m,:] = -(2*m-1)*sx*Plm[m-1,m-1,:]

In [6]:
# Now do m=ell-1
for m in range(1,Nl-1):
    Plm[m+1,m,:] = (2*m+1)*xx*Plm[m,m,:]

In [7]:
# Finally fill in ell>m+1:
for m in range(0,Nl-1):
    for ell in range(m+2,Nl):
        Plm[ell,m,:] = (2*ell-1)*xx*Plm[ell-1,m,:]-(ell+m-1)*Plm[ell-2,m,:]
        Plm[ell,m,:]/= (ell-m)

In [8]:
# Now let's compare to the library version.  Compute a
# table in the same format from SciPy.  Note SciPy has
# the ell,m indices reversed from our convention and does
# all ell,m for a single x.
from scipy.special import lpmn
#
Pmn = np.zeros( (Nl+1,Nl+1,Nx) )
for i,x in enumerate(xx):
    val,deriv  = lpmn(Nl,Nl,x)
    Pmn[:,:,i] = val.T

In [9]:
# Just cross-check some values for sanity.
for ell in [1,5,10,25,30,40]:
    for m in [1,5,10,15,25,40]:
        if m<=ell:
            lbs,mys = "",""
            for i in range(Nx): lbs+= " {:12.4e}".format(Pmn[ell,m,i])
            for i in range(Nx): mys+= " {:12.4e}".format(Plm[ell,m,i])
            print("({:2d},{:2d}):".format(ell,m))
            print("\t"+lbs)
            print("\t"+mys)

( 1, 1):
	  -9.9969e-01  -9.5394e-01  -8.6603e-01  -7.1414e-01  -2.2220e-01
	  -9.9969e-01  -9.5394e-01  -8.6603e-01  -7.1414e-01  -2.2220e-01
( 5, 1):
	  -1.8580e+00   1.6080e-01   1.9283e+00   1.0952e+00  -2.7784e+00
	  -1.8580e+00   1.6080e-01   1.9283e+00   1.0952e+00  -2.7784e+00
( 5, 5):
	  -9.4352e+02  -7.4651e+02  -4.6035e+02  -1.7553e+02  -5.1192e-01
	  -9.4352e+02  -7.4651e+02  -4.6035e+02  -1.7553e+02  -5.1192e-01
(10, 1):
	  -6.6924e-01   1.2310e-01   2.0067e+00  -2.9685e+00  -5.5867e+00
	  -6.6924e-01   1.2310e-01   2.0067e+00  -2.9685e+00  -5.5867e+00
(10, 5):
	  -6.2798e+03  -9.2725e+03   3.0086e+04  -2.0321e+04  -1.2962e+03
	  -6.2798e+03  -9.2725e+03   3.0086e+04  -2.0321e+04  -1.2962e+03
(10,10):
	   6.5269e+08   4.0857e+08   1.5537e+08   2.2590e+07   1.9213e+02
	   6.5269e+08   4.0857e+08   1.5537e+08   2.2590e+07   1.9213e+02
(25, 1):
	  -3.2393e+00  -3.6647e-01  -3.0879e+00  -2.9323e+00   8.2600e+00
	  -3.2393e+00  -3.6647e-01  -3.0879e+00  -2.9323e+00   8.2600e+00

Note that the size of these polynomials, especially when $\ell\approx m$, is getting very large.  For this reason we likely want to include the normalization as in the spherical harmonics.

For example if $\bar{P}_\ell^m=\sqrt{(\ell-m)!/(\ell+m)!}\,P_\ell^m$ then
\begin{align}
    \bar{P}_m^m &= -(2m-1)\sqrt{1-x^2} \sqrt{ \frac{(2m-2)!}{(2m)!} }\ \bar{P}_{m-1}^{m-1} \\
    &= -\sqrt{1-x^2} \sqrt{ \frac{2m-1}{2m} }\ \bar{P}_{m-1}^{m-1} \\
    &= -\sqrt{1-x^2} \sqrt{ 1-(2m)^{-1} } \ \bar{P}_{m-1}^{m-1}
\end{align}
and note the problematic factorial growth is gone.  For large values of $m$ we could use
\begin{equation}
    \sqrt{1-(2m)^{-1}}\simeq 1 - \frac{1}{4m} - \frac{1}{32m^2} - \frac{1}{128m^3} - \frac{5}{2048m^4}
\end{equation}
which for $m>20$ is accurate to 10 decimal places.  Similarly
\begin{align}
    \bar{P}_{m+1}^m &= (2m+1)x\ \sqrt{ \frac{1!}{(2m+1)!} \frac{(2m)!}{1!} } \bar{P}_m^m \\
    &= \sqrt{2m+1}\, x\ \bar{P}_m^m
\end{align}
Finally
\begin{align}
    (\ell-m) \bar{P}_{\ell}^m(x) &= \sqrt{\frac{(\ell-m)!}{(\ell+m)!}}\left[ (2\ell-1) x P_{\ell-1}^m(x) - (\ell+m-1) P_{\ell-2}^m(x) \right] \\
    &= \sqrt{\frac{(\ell-m)!}{(\ell+m)!}} (2\ell-1) x \sqrt{\frac{(\ell+m-1)!}{(\ell-m-1)!}} \, \bar{P}_{\ell-1}^m(x) \nonumber \\
    &- \sqrt{\frac{(\ell-m)!}{(\ell+m)!}} (\ell+m-1) \sqrt{\frac{(\ell+m-2)!}{(\ell-m-2)!}} \, \bar{P}_{\ell-2}^m(x) \\
    &= \sqrt{\frac{(\ell-m)}{(\ell+m)}}\, (2\ell-1) x  \bar{P}_{\ell-1}^m(x)  \nonumber \\
    &- \sqrt{\frac{(\ell-m)(\ell-m-1)}{(\ell+m)(\ell+m-1)}} \, (\ell+m-1)  \bar{P}_{\ell-2}^m(x) \\
    \bar{P}_\ell^m
    &= \sqrt{\frac{(\ell-m)}{(\ell+m)}}\, \frac{2\ell-1}{\ell-m}\ x\,  \bar{P}_{\ell-1}^m(x)  \nonumber \\
    &- \sqrt{\frac{(\ell-m)(\ell-m-1)}{(\ell+m)(\ell+m-1)}} \, \frac{\ell+m-1}{\ell-m}\  \bar{P}_{\ell-2}^m(x) 
\end{align}

In [10]:
# Let's see how well this works.
Plm = np.zeros( (Nl,Nl,Nx) )
# First we do the m=0 case, which is the same as before.
Plm[0,0,:] = np.ones_like(xx)
Plm[1,0,:] = xx.copy()
for ell in range(2,Nl):
    Plm[ell,0,:] = (2*ell-1)*xx*Plm[ell-1,0,:]-(ell-1)*Plm[ell-2,0,:]
    Plm[ell,0,:]/= float(ell)

In [11]:
# Now we fill in m>0.
# To keep the recurrences stable, we treat "high m" and "low m" separately.
# Start with the highest value of m allowed:
for m in range(1,Nl):
    Plm[m,m,:] = -np.sqrt(1.0-1./(2*m))*sx*Plm[m-1,m-1,:]

In [12]:
# Now do m=ell-1
for m in range(1,Nl-1):
    Plm[m+1,m,:] = np.sqrt(2*m+1.)*xx*Plm[m,m,:]

In [13]:
# Finally fill in ell>m+1:
for m in range(0,Nl-1):
    for ell in range(m+2,Nl):
        fact1,fact2  = np.sqrt( (ell-m)*1./(ell+m) ),np.sqrt( (ell-m-1.)/(ell+m-1.) )
        Plm[ell,m,:] = (2*ell-1)*xx*Plm[ell-1,m,:]*fact1-(ell+m-1)*Plm[ell-2,m,:]*fact1*fact2
        Plm[ell,m,:]/= (ell-m)

In [14]:
# And let's put in the (2ell+1)/4pi normalization as well.
for ell in range(Nl):
    Plm[ell,:,:] *= np.sqrt( (2*ell+1)/4./np.pi )

In [15]:
# Now let's compare to the library version.  Compute a
# table in the same format from SciPy.  Note SciPy has
# the ell,m indices and theta,phi arguments reversed!
from scipy.special import sph_harm
#
Ylm = np.zeros( (Nl,Nl,Nx) )
for ell in range(Nl):
    for m in range(ell+1):
        for i,x in enumerate(xx):
            theta        = np.arccos(x)
            Ylm[ell,m,i] = np.real( sph_harm(m,ell,0,theta) )

In [16]:
# Just cross-check some values for sanity.
for ell in [1,5,10,25,30,40,95]:
    for m in [1,5,10,15,25,40,75,95]:
        if m<=ell:
            lbs,mys = "",""
            for i in range(Nx): lbs+= " {:12.4e}".format(Ylm[ell,m,i])
            for i in range(Nx): mys+= " {:12.4e}".format(Plm[ell,m,i])
            print("({:2d},{:2d}):".format(ell,m))
            print("\t"+lbs)
            print("\t"+mys)

( 1, 1):
	  -3.4539e-01  -3.2958e-01  -2.9921e-01  -2.4673e-01  -7.6770e-02
	  -3.4539e-01  -3.2958e-01  -2.9921e-01  -2.4673e-01  -7.6770e-02
( 5, 1):
	  -3.1738e-01   2.7467e-02   3.2938e-01   1.8708e-01  -4.7460e-01
	  -3.1738e-01   2.7467e-02   3.2938e-01   1.8708e-01  -4.7460e-01
( 5, 5):
	  -4.6341e-01  -3.6664e-01  -2.2610e-01  -8.6212e-02  -2.5143e-04
	  -4.6341e-01  -3.6664e-01  -2.2610e-01  -8.6212e-02  -2.5143e-04
(10, 1):
	  -8.2488e-02   1.5172e-02   2.4734e-01  -3.6589e-01  -6.8860e-01
	  -8.2488e-02   1.5172e-02   2.4734e-01  -3.6589e-01  -6.8860e-01
(10, 5):
	  -7.7766e-02  -1.1483e-01   3.7257e-01  -2.5165e-01  -1.6052e-02
	  -7.7766e-02  -1.1483e-01   3.7257e-01  -2.5165e-01  -1.6052e-02
(10,10):
	   5.4094e-01   3.3862e-01   1.2877e-01   1.8722e-02   1.5924e-07
	   5.4094e-01   3.3862e-01   1.2877e-01   1.8722e-02   1.5924e-07
(25, 1):
	  -2.5596e-01  -2.8957e-02  -2.4400e-01  -2.3170e-01   6.5268e-01
	  -2.5596e-01  -2.8957e-02  -2.4400e-01  -2.3170e-01   6.5268e-01

Note that the library version is producing NaNs for $\ell=m=95$, but also that for high $\ell$ and $m$ the spherical harmonics are quite small for $x\approx 1$.

# The End